In [21]:
import torch
x = torch.tensor([[5.5,3],[1,2],[5,0]])
print(x)
y = torch.randn_like(x,dtype=torch.float)
print(y)
print(x+y,y.size())

tensor([[5.5000, 3.0000],
        [1.0000, 2.0000],
        [5.0000, 0.0000]])
tensor([[ 0.7544, -0.9401],
        [ 0.1931,  1.5394],
        [-1.3066, -0.5627]])
tensor([[ 6.2544,  2.0599],
        [ 1.1931,  3.5394],
        [ 3.6934, -0.5627]]) torch.Size([3, 2])


In [31]:
# 如果将其属性 .requires_grad 设置为 True，则会开始跟踪针对 tensor 的所有操作
x = torch.ones(3, 2, requires_grad=True)
# print(x)
y = x + 2
# print(y,y.grad_fn)

z = y*y*3
out = z.mean()
print(z, out)
out.backward()
print(x.grad)

# a = torch.randn(2, 2)
# a = ((a * 3) / (a - 1))
# print(a.requires_grad)
# a.requires_grad_(True)
# print(a.requires_grad)
# b = (a * a).sum()
# print(b,b.grad_fn)

tensor([[27., 27.],
        [27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)
tensor([[3., 3.],
        [3., 3.],
        [3., 3.]])


In [60]:
x = torch.randn(3, requires_grad=True)
print(x)
y = x * 2
while y.data.norm() < 1000:
  print(y.data.norm())
  y = y * 2
print(y)
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)


tensor([-0.4303,  1.1554,  1.2227], requires_grad=True)
tensor(3.4728)
tensor(6.9456)
tensor(13.8912)
tensor(27.7824)
tensor(55.5648)
tensor(111.1296)
tensor(222.2592)
tensor(444.5183)
tensor(889.0367)
tensor([-440.6380, 1183.1306, 1252.0323], grad_fn=<MulBackward0>)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [63]:
# 神经网络
import torch
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
  
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
torch.Size([6, 1, 5, 5])


In [78]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)
# print(input,input.size)

tensor([[-0.0414, -0.0734, -0.1011,  0.0233, -0.1059,  0.0061, -0.1027, -0.0664,
         -0.1070, -0.0773]], grad_fn=<AddmmBackward>)


In [79]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [87]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)# make it the same shape as output
criterion = nn.MSELoss()
print(output,target)
loss = criterion(output,target)
print(loss)


tensor([[-0.0414, -0.0734, -0.1011,  0.0233, -0.1059,  0.0061, -0.1027, -0.0664,
         -0.1070, -0.0773]], grad_fn=<AddmmBackward>) tensor([[ 1.6366, -0.4556, -0.1748, -0.3339, -0.5980, -2.2791, -0.2987, -1.1109,
          1.0826, -0.3172]])
tensor(1.1161, grad_fn=<MseLossBackward>)
